In [ ]:
import numpy as np
import pandas as pd
import re
from google.colab import drive
drive.mount("/drive")

Mounted at /drive


In [ ]:
df = pd.read_excel('/drive/My Drive/6SP/logs.xlsx')
df_desc = df[df['Description'].notnull()] # remove rows with empty description field

In [ ]:
def userId(desc):
    return desc.split()[4][1:-1]

In [ ]:
df_desc['User_id'] = df_desc['Description'].apply(userId)
#df_desc.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_new = df_desc[df_desc['Event context'].notnull()] # remove rows with empty event context field
df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id
0,"28/02/22, 10:56",Course: GP106-2021 : Computing,Logs,Log report viewed,The user with id '2614' viewed the log report ...,web,2614
1,"28/02/22, 10:56",Course: GP106-2021 : Computing,System,Course viewed,The user with id '2614' viewed the course with...,web,2614
2,"28/02/22, 10:04",Course: GP106-2021 : Computing,System,Course viewed,The user with id '3740' viewed the course with...,web,3740
3,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Course viewed,The user with id '30' viewed the course with i...,web,30
4,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Role assigned,The user with id '30' assigned the role with i...,web,30


In [ ]:
# function to extract different event context types  
def extractEvent(eventContext):
  return eventContext.split(":")[0]

In [ ]:
df_new['Event'] = df_new['Event context'].apply(extractEvent)
#df_new.head()

In [ ]:
# extract only log records with event context type of "Label" and "File"
df_new = df_new[df_new.Event.isin(["Label","File"])]
#df_new.head()

In [ ]:
# function to extract different event context types  2nd value given
def extractEventName(eventContext):
  return eventContext.split(":")[1]

In [ ]:
df_new['EventName'] = df_new['Event context'].apply(extractEventName)
df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id,Event,EventName
14,"28/02/22, 06:59",File: GP106 Project Specification,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,File,GP106 Project Specification
15,"28/02/22, 06:59",Label: GP106 Project\n\nInformation about the ...,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Project\n\nInformation about the projec...
16,"28/02/22, 06:59",Label: GP106 Labs: (copy),System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Labs
21,"28/02/22, 05:51",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs
24,"28/02/22, 05:40",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs


In [ ]:
# function to extract status from event name field
def extractStatus(eventName):
  return eventName.split()[2]

In [ ]:
# function to extract date
def extractDate(date_time):
  return date_time.split(",")[0]

In [ ]:
df_new['Status'] = df_new['Event name'].apply(extractStatus)
df_new['Date'] = df_new['Time'].apply(extractDate)
#df_new.head()

In [ ]:
# function to give type from Event context field
def extractType(eventContext):
  if (eventContext.split(":")[1][1:8] == 'Lecture') & (len(eventContext.split(":")[1]) < 15) :
    return "Video"
  elif (eventContext.split()[1][0:7] == 'Handout') :
    return "PDF"
  else:
    return "Other"

In [ ]:
df_new['Type'] = df_new['Event context'].apply(extractType)
df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id,Event,EventName,Status,Date,Type
14,"28/02/22, 06:59",File: GP106 Project Specification,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,File,GP106 Project Specification,completion,28/02/22,Other
15,"28/02/22, 06:59",Label: GP106 Project\n\nInformation about the ...,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Project\n\nInformation about the projec...,completion,28/02/22,Other
16,"28/02/22, 06:59",Label: GP106 Labs: (copy),System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Labs,completion,28/02/22,Other
21,"28/02/22, 05:51",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,viewed,28/02/22,Other
24,"28/02/22, 05:40",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,viewed,28/02/22,Other


In [ ]:
# function to extract module Id from description field
def extractModuleID(desc):
  if (desc.split()[5] == 'updated'):
    return desc.split()[-7]
  else:
    return desc.split()[-1]

In [ ]:
df_new['Module_id'] = df_new['Description'].apply(extractModuleID)
df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id,Event,EventName,Status,Date,Type,Module_id
14,"28/02/22, 06:59",File: GP106 Project Specification,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,File,GP106 Project Specification,completion,28/02/22,Other,'49119'
15,"28/02/22, 06:59",Label: GP106 Project\n\nInformation about the ...,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Project\n\nInformation about the projec...,completion,28/02/22,Other,'48654'
16,"28/02/22, 06:59",Label: GP106 Labs: (copy),System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Labs,completion,28/02/22,Other,'48652'
21,"28/02/22, 05:51",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,viewed,28/02/22,Other,'34437'.
24,"28/02/22, 05:40",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,viewed,28/02/22,Other,'34437'.


In [ ]:
df_new = df_new.set_index('Status')
df_new = df_new.rename(index={'completion':'completed'})
#df_new.head()

In [ ]:
df_new=df_new.reset_index()
#df_new.head()

In [ ]:
df_new.rename(columns={'EventName': 'Material_name'}, inplace=True)
df_new.head()

,Status,Time,Event context,Component,Event name,Description,Origin,User_id,Event,Material_name,Date,Type,Module_id
0,completed,"28/02/22, 06:59",File: GP106 Project Specification,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,File,GP106 Project Specification,28/02/22,Other,'49119'
1,completed,"28/02/22, 06:59",Label: GP106 Project\n\nInformation about the ...,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Project\n\nInformation about the projec...,28/02/22,Other,'48654'
2,completed,"28/02/22, 06:59",Label: GP106 Labs: (copy),System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Labs,28/02/22,Other,'48652'
3,viewed,"28/02/22, 05:51",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,28/02/22,Other,'34437'.
4,viewed,"28/02/22, 05:40",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,28/02/22,Other,'34437'.


In [ ]:
#df_learningMaterials.to_csv("Learning_Materials_1.csv")

In [ ]:
# function to remove other files which are not learning materials
def extractOther(eventContext):
  type = eventContext.split()[1][0:7]
  if(eventContext.split(":")[0] == 'Label'):
    if (type == 'Lecture') | (type == 'Lab') | (type == 'Old'):
      return 'True'
    else:
      return 'False'
  else:
    if (type == 'Dataset') | (type == 'Handout'):
      return "True"
    else:
      return "False"

In [ ]:
df_new['Remove'] = df_new['Event context'].apply(extractOther)
df_new.head()

,Status,Time,Event context,Component,Event name,Description,Origin,User_id,Event,Material_name,Date,Type,Module_id,Remove
0,completed,"28/02/22, 06:59",File: GP106 Project Specification,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,File,GP106 Project Specification,28/02/22,Other,'49119',False
1,completed,"28/02/22, 06:59",Label: GP106 Project\n\nInformation about the ...,System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Project\n\nInformation about the projec...,28/02/22,Other,'48654',False
2,completed,"28/02/22, 06:59",Label: GP106 Labs: (copy),System,Course activity completion updated,The user with id '3489' updated the completion...,web,3489,Label,GP106 Labs,28/02/22,Other,'48652',False
3,viewed,"28/02/22, 05:51",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,28/02/22,Other,'34437'.,False
4,viewed,"28/02/22, 05:40",File: Group List for Labs,File,Course module viewed,The user with id '3638' viewed the 'resource' ...,web,3638,File,Group List for Labs,28/02/22,Other,'34437'.,False


In [ ]:
df_new = df_new[df_new.Remove != 'False']

df_new.head()

,Status,Time,Event context,Component,Event name,Description,Origin,User_id,Event,Material_name,Date,Type,Module_id,Remove
44,viewed,"24/02/22, 00:04",File: Handout13 : Matrix Manipulation,File,Course module viewed,The user with id '3391' viewed the 'resource' ...,web,3391,File,Handout13,24/02/22,PDF,'42299'.,True
45,viewed,"24/02/22, 00:01",File: Handout 14: Plotting Graphs using Matplo...,File,Course module viewed,The user with id '3391' viewed the 'resource' ...,web,3391,File,Handout 14,24/02/22,PDF,'43550'.,True
62,viewed,"20/02/22, 19:26",File: Handout 14: Plotting Graphs using Matplo...,File,Course module viewed,The user with id '3466' viewed the 'resource' ...,web,3466,File,Handout 14,20/02/22,PDF,'43550'.,True
87,viewed,"19/02/22, 21:07",File: Handout 7: Control Structures,File,Course module viewed,The user with id '3648' viewed the 'resource' ...,web,3648,File,Handout 7,19/02/22,PDF,'38179'.,True
94,viewed,"19/02/22, 20:38",File: Handout 1: Introduction,File,Course module viewed,The user with id '3754' viewed the 'resource' ...,web,3754,File,Handout 1,19/02/22,PDF,'33243'.,True


In [ ]:
df_learningMaterials = df_new[['User_id','Material_name','Module_id','Type','Status','Date']]
df_learningMaterials.head()

,User_id,Material_name,Module_id,Type,Status,Date
44,3391,Handout13,'42299'.,PDF,viewed,24/02/22
45,3391,Handout 14,'43550'.,PDF,viewed,24/02/22
62,3466,Handout 14,'43550'.,PDF,viewed,20/02/22
87,3648,Handout 7,'38179'.,PDF,viewed,19/02/22
94,3754,Handout 1,'33243'.,PDF,viewed,19/02/22


In [ ]:
df_learningMaterials.to_csv("Learning_Materials.csv")

In [ ]:
df_summary = df_learningMaterials.groupby(['Type','Status']).size().reset_index().groupby(['Type','Status'])[[0]].max()
df_summary

0
Type  Status          
Other completed   1651
      created        9
      updated        8
      viewed      3135
PDF   completed   3367
      created       14
      updated        4
      viewed     12973
Video completed   2074
      created       15
      updated       12

In [ ]:
df_summary.to_csv("LearningMaterial_summary.csv")